# Lab Assignment 09: Recurrent Network Architectures

Conrad Appel, Erik Gabrielsen, Danh Nguyen

### Dataset Selection

Select a dataset identically to lab two. That is, the dataset must be text data (or time series sequence). In terms of generalization performance, it is helpful to have a large dataset of similar sized text documents. It is fine to perform binary classification or multi-class classification. The classification can be "many-to-one" or "many-to-many" sequence classification, whichever you feel more comfortable with. 

### Preparation (40 points total)
[20 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).   

[10 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

[10 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your cross validation method is a realistic mirroring of how an algorithm would be used in practice. 

### Modeling (50 points total)

[25 points] Investigate at least two different recurrent network architectures (perhaps LSTM and GRU). Adjust hyper-parameters of the networks as needed to improve generalization performance. 

[25 points] Use the method of cross validation and evaluation metric that you argued for at the beginning of the lab. Visualize the best results of the RNNs.   

### Exceptional Work (10 points total)
You have free reign to provide additional analyses.

One idea: Use more than a single chain of LSTMs or GRUs (i.e., use multiple parallel chains). 

Another Idea: Try to create a RNN for generating novel text. 

In [1]:
import pandas as pd
import numpy as np
import os
from scipy.misc import imread
import matplotlib.pyplot as plt
import random
import warnings
from keras.datasets import reuters
from keras.preprocessing import sequence
from keras import utils


Using TensorFlow backend.


In [2]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=top_words,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [3]:
# truncate and pad input sequences
max_art_length = 500
NUM_CLASSES = 46
X_train = sequence.pad_sequences(X_train, maxlen=max_art_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_art_length)

y_train_ohe = utils.to_categorical(y_train, NUM_CLASSES)
y_test_ohe = utils.to_categorical(y_test, NUM_CLASSES)

In [4]:
print(type(X_train),X_train.shape)
print(type(X_train[0]),X_train[0].shape)
print('Vocabulary size:', np.max(X_train))
print(y_train.shape, np.min(y_train), np.max(y_train))

<class 'numpy.ndarray'> (8982, 500)
<class 'numpy.ndarray'> (500,)
Vocabulary size: 4999
(8982,) 0 45


In [5]:
from keras.models import Sequential, Input, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers.embeddings import Embedding

EMBED_SIZE = 50
rnns = []
input_holder = Input(shape=(X_train.shape[1], ))
shared_embed = Embedding(top_words, 
                EMBED_SIZE, 
                input_length=max_art_length)(input_holder)

for func in [SimpleRNN, LSTM, GRU]:
    
    x = func(50,dropout=0.2, recurrent_dropout=0.2)(shared_embed)
    x = Dense(NUM_CLASSES, activation='sigmoid')(x)
    rnn=Model(inputs=input_holder,outputs=x)
    rnn.compile(loss='categorical_crossentropy', 
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
    print(rnn.summary())
    rnns.append(rnn)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 50)                5050      
_________________________________________________________________
dense_1 (Dense)              (None, 46)                2346      
Total params: 257,396
Trainable params: 257,396
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________

In [6]:
for rnn, name in zip(rnns,['lstm','gru']):
    print('=======',name,'========')
    rnn.fit(X_train, y_train_ohe, epochs=10, batch_size=64, validation_data=(X_test, y_test_ohe))

======= lstm ========
Train on 8982 samples, validate on 2246 samples
Epoch 1/10
8982/8982 [==============================] - 20s - loss: 3.3987 - acc: 0.2219 - val_loss: 2.6831 - val_acc: 0.3606
Epoch 2/10
8982/8982 [==============================] - 40s - loss: 2.4870 - acc: 0.3484 - val_loss: 2.4345 - val_acc: 0.3620
Epoch 3/10
8982/8982 [==============================] - 39s - loss: 2.4148 - acc: 0.3517 - val_loss: 2.4133 - val_acc: 0.3620
Epoch 4/10
8982/8982 [==============================] - 29s - loss: 2.3833 - acc: 0.3517 - val_loss: 2.3690 - val_acc: 0.3620
Epoch 5/10
8982/8982 [==============================] - 19s - loss: 2.3443 - acc: 0.3570 - val_loss: 2.3069 - val_acc: 0.3776
Epoch 6/10
8982/8982 [==============================] - 18s - loss: 2.2802 - acc: 0.3793 - val_loss: 2.2632 - val_acc: 0.3900
Epoch 7/10
8982/8982 [==============================] - 20s - loss: 2.2243 - acc: 0.4037 - val_loss: 2.2385 - val_acc: 0.4020
Epoch 8/10
8982/8982 [==========================

In [7]:
for rnn, name in zip(rnns,['lstm','gru']):
    print('Round 2 training,',name,'========')
    rnn.fit(X_train, y_train_ohe, epochs=10, batch_size=64, validation_data=(X_test, y_test_ohe))

Round 2 training, lstm ========
Train on 8982 samples, validate on 2246 samples
Epoch 1/10
8982/8982 [==============================] - 27s - loss: 2.0007 - acc: 0.4527 - val_loss: 1.9435 - val_acc: 0.4622
Epoch 2/10
8982/8982 [==============================] - 34s - loss: 1.9008 - acc: 0.4743 - val_loss: 1.8968 - val_acc: 0.4666
Epoch 3/10
8982/8982 [==============================] - 34s - loss: 1.8491 - acc: 0.4892 - val_loss: 1.8937 - val_acc: 0.4684
Epoch 4/10
8982/8982 [==============================] - 33s - loss: 1.8269 - acc: 0.4991 - val_loss: 1.8969 - val_acc: 0.4715
Epoch 5/10
8982/8982 [==============================] - 30s - loss: 1.7952 - acc: 0.5038 - val_loss: 1.8895 - val_acc: 0.4751
Epoch 6/10
8982/8982 [==============================] - 29s - loss: 1.7691 - acc: 0.5157 - val_loss: 1.8824 - val_acc: 0.4684
Epoch 7/10
8982/8982 [==============================] - 26s - loss: 1.7447 - acc: 0.5269 - val_loss: 1.9013 - val_acc: 0.4751
Epoch 8/10
8982/8982 [================

In [8]:
for rnn, name in zip(rnns,['lstm','gru']):
    rnn.save(name+'.h5')